In [1]:
import matplotlib

from keras.preprocessing.image import ImageDataGenerator,img_to_array
from keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from model.small_vggnet import SmallerVggNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import random
import cv2
import os

Using TensorFlow backend.


In [2]:
EPOCH = 30
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96,96,3)



##### Data preprocessing


In [3]:
imagePaths = sorted(list(paths.list_images(r'C:\Users\prith\Multi_Label_classification\dataset')))
random.seed(42)
random.shuffle(imagePaths)

data = []
labels = []

for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    
    l = label = imagePath.split(os.path.sep)[-2].split(" ")
    labels.append(l)
    
data = np.array(data,dtype='float') / 255.0
labels = np.array(labels)


In [4]:
#label binarizer

print("class labels:")
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

for i,label in enumerate(mlb.classes_):
    print("{}.{}".format(i+1,label))

class labels:
1.black_jeans
2.black_shirt
3.blue_jeans
4.blue_shirt


In [5]:
(xtrain,xtest,ytrain,ytest) = train_test_split(data,labels,test_size=0.2,random_state=42)

In [6]:
aug = ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,
                        shear_range=0.2,zoom_range=0.2,fill_mode='nearest')

##### model training

In [ ]:
model = SmallerVggNet.build(width=IMAGE_DIMS[1],height=IMAGE_DIMS[0],depth=IMAGE_DIMS[2],classes=len(mlb.classes_),finalAct='sigmoid')
opt = Adam(learning_rate=INIT_LR,decay=INIT_LR/EPOCH)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()
history = model.fit(x=aug.flow(xtrain,ytrain,batch_size=BS),validation_data=(xtest,ytest),steps_per_epoch=len(xtrain)//BS,epochs=EPOCH,verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)       

 3/26 [==>...........................] - ETA: 1s - loss: 0.1255 - accuracy: 0.9579

##### saving model and label binarizer to disk

In [ ]:
model.save_weights('multiclass_img_classifier.h5')

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCH
plt.plot(np.arange(0, N), history.history["loss"],'r--', label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"],'b--', label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"],'r', label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"],'b', label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="best")
plt.savefig(r'C:\Users\prith\Multi_Label_classification\plot.png')
plt.show()

In [ ]:
import imutils

In [ ]:
img = r'C:\Users\prith\Multi_Label_classification\examples\6.jpg'
image = cv2.imread(img)
image = cv2.resize(image,(96,96))
image = image.astype('float')/255.0
image = img_to_array(image)
image = np.expand_dims(image,axis=0)

In [ ]:
proba = model.predict(image)[0]

In [ ]:
idxs = np.argsort(proba)[::-1][:2]

In [ ]:
img = cv2.imread(img)
output = imutils.resize(img,width=400)

In [ ]:
from PIL import Image
op = r'C:\Users\prith\Multi_Label_classification\examples\6.jpg'

In [ ]:
for (i,j) in enumerate(idxs):
    label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
    cv2.putText(output, label, (10, (i * 30) + 25),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
for (label,p) in zip(mlb.classes_,proba):
    print("{}: {:.2f}%".format(label, p * 100))
    
Image.open(op)